#### I. Imports et chargement

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
XLSX_URL = "https://huggingface.co/datasets/flodussart/getaround_xls_certif/resolve/main/get_around_delay_analysis.xlsx"
df = pd.read_excel(XLSX_URL, sheet_name=0)

CSV_URL = "https://huggingface.co/datasets/flodussart/getaround_pricing_project/resolve/main/get_around_pricing_project.csv"
dataset_pricing = pd.read_csv(CSV_URL)

#### II. Constantes

In [22]:
# Outlier handling for delays (minutes)
CLIP_MIN, CLIP_MAX = -500, 1000

COL_CHECKIN = "checkin_type"  
COL_GAP = "time_delta_with_previous_rental_in_minutes"  
COL_RENTAL = "rental_id"  
COL_PREV_ID = "previous_ended_rental_id"  
COL_NEXT_ID = "next_rental_id"
COL_DELAY_RAW = "delay_at_checkout_in_minutes"
COL_STATE = "state"
COL_CAR = "car_id"
# Define color palette for check-in types
COLOR_CI = {
    "mobile": "#4cc9f0",  # Light blue for mobile check-in
    "connect": "#ffb703",  # Orange for connect check-in
}

# Visualization and threshold parameters
NBINS = 60  
THRESHOLDS = [60, 120]   # 1h / 2h markers
THRESHOLDS_ROI = np.arange(0, 721, 15)  # 0 to 12h in 15-min steps
SEUILS = list(range(0, 401, 10))  # 0 to 400 min by step of 10
SEUIL_METRIQUE = 60  # Metric threshold (e.g., 1 hour buffer)
T_DISPLAY = 60  # Time display unit for plots (in minutes)

#### III. Périmètre d'analyse (retards & locations terminées)

In [4]:
# Build analysis tables (clipped delay + ended rentals only)
df_base = df.copy()

df_base["delay_clipped"] = df_base[COL_DELAY_RAW].clip(
    CLIP_MIN, CLIP_MAX
)

df_ended = df_base.loc[df_base[COL_STATE].eq("ended")].copy()

# Quick sanity check
print(
    f"Base: {len(df_base):,} rows | Ended: {len(df_ended):,} | "
    f"Missing delay (ended): {df_ended[COL_DELAY_RAW].isna().sum():,}"
)

Base: 21,310 rows | Ended: 18,045 | Missing delay (ended): 1,700


<span style="font-size:0.85em">

Préparation de la base canonique utilisée dans tout le reste de l’analyse.

Étapes principales :
- Création d’une copie de travail du dataset brut afin de préserver les données originales.
- Construction d’une variable `delay_clipped` en bornant le retard brut (`delay_at_checkout_in_minutes`) afin de limiter l’impact des valeurs extrêmes sur les graphiques et indicateurs.
- Restriction du périmètre aux locations effectivement terminées (`state = "ended"`), seules pour lesquelles un retard est interprétable.
- Affichage d’un sanity check : taille des jeux de données et nombre de retards manquants.

Le DataFrame `df_ended` constitue la source de vérité pour toutes les analyses suivantes (distribution des retards, conflits entre locations, propagation, ROI, scénarios business).
</span>

#### IV. Helpers : Fonctions utilitaires (scope / gap / location suivante)

In [ ]:
def get_scoped(df_ended: pd.DataFrame, scope: str = "all") -> pd.DataFrame:
    """Filter ended rentals by check-in scope: all / connect / mobile."""
    if scope == "all":
        return df_ended
    if scope in {"connect", "mobile"}:
        return df_ended.loc[df_ended[COL_CHECKIN].eq(scope)]
    raise ValueError("scope must be one of: 'all', 'connect', 'mobile'")


def make_df_gap(df_ended: pd.DataFrame) -> pd.DataFrame:
    """Keep ended rentals with delay+gap available and flag conflicts (delay > gap)."""
    out = df_ended.loc[
        df_ended[COL_CHECKIN].isin(["mobile", "connect"])
        & df_ended["delay_clipped"].notna()
        & df_ended[COL_GAP].notna(),
        [COL_RENTAL, COL_CAR, COL_CHECKIN, "delay_clipped", COL_GAP],
    ].copy()

    out = out.rename(columns={COL_GAP: "gap"})
    out["was_conflict"] = out["delay_clipped"] > out["gap"]
    return out


def make_df_next(df_ended: pd.DataFrame) -> pd.DataFrame:
    """Attach next rental_id for each ended rental via previous_ended_rental_id."""
    needed = {COL_RENTAL, COL_PREV_ID}
    if not needed.issubset(df_ended.columns):
        out = df_ended.copy()
        out[COL_NEXT_ID] = pd.NA
        return out

    # left join: current rental_id is the "next" of previous_ended_rental_id
    next_map = df_ended.loc[:, [COL_RENTAL, COL_PREV_ID]].rename(
        columns={COL_RENTAL: COL_NEXT_ID}
    )

    out = df_ended.merge(
        next_map,
        left_on=COL_RENTAL,
        right_on=COL_PREV_ID,
        how="left",
        suffixes=("", "_drop"),
    ).drop(columns=[COL_PREV_ID + "_drop"])
    return out

<span style="font-size:0.85em">

Définition de 3 fonctions réutilisées dans tout le notebook pour standardiser le périmètre et préparer des tables prêtes à tracer.

1) `get_scoped(df_ended, scope)`
Objectif : filtrer `df_ended` selon le type de check-in.
- `scope="all"` : ne filtre pas (retourne tout `df_ended`)
- `scope="connect"` ou `"mobile"` : conserve uniquement les lignes où `checkin_type == scope`
- Sinon : lève une erreur (évite les scopes incohérents)

Permet de comparer facilement *ALL vs CONNECT vs MOBILE* avec la même logique.

2) `make_df_gap(df_ended)`
Objectif : construire une table “gap” propre pour l’analyse des conflits entre deux locations.
- Ne garde que les check-ins `mobile/connect`
- Ne garde que les lignes où `delay_clipped` et le `gap` sont connus (non-NaN)
- Renomme `time_delta_with_previous_rental_in_minutes` en `gap`
- Ajoute `was_conflict = (delay_clipped > gap)` :
  > conflit historique = le retard dépasse le temps disponible entre 2 locations

Sortie : un DataFrame minimal (`rental_id`, `car_id`, `checkin_type`, `delay_clipped`, `gap`, `was_conflict`) utilisé pour calculer *% masquées / % évités*.

3) `make_df_next(df_ended)`
Objectif : rattacher, pour chaque location, la location suivante (si elle existe) afin d’étudier la propagation des retards.
- Construit une table `next_map` : `(previous_ended_rental_id → next_rental_id)`
- Merge sur `rental_id` pour ajouter une colonne `next_rental_id`
- Si les colonnes nécessaires n’existent pas : crée `next_rental_id = NA` (fallback sûr)

Sert ensuite à créer des paires `A → B` et comparer `delay(A)` vs `delay(B)`.

> Bénéfice global : ces helpers rendent le notebook plus lisible et évitent de répéter du code de filtrage/feature engineering à chaque partie.
</span>


#### V. Scope : Périmètre d'analyse

In [23]:
# Define a single scope for consistency across all sections
SCOPE = "all"  # options: 'all' | 'connect' | 'mobile'

# Apply the chosen scope:
# - Section 1: Delay distribution analysis
# - Section 3: Gap-based curves (masked / avoided rentals)
# - Section 2: Propagation to the next rental
df_scope = get_scoped(df_ended, SCOPE)
df_gap = make_df_gap(df_scope)
df_next = make_df_next(df_scope)

# Sanity check: display key row counts for each dataset
print(
    f"Scope '{SCOPE}': "
    f"ended rows={len(df_scope):,} | "
    f"gap rows={len(df_gap):,} | "
    f"with next={df_next[COL_NEXT_ID].notna().sum():,}"
)

Scope 'all': ended rows=18,045 | gap rows=1,515 | with next=1,612


<span style="font-size:0.85em">

Application d'un périmètre d’analyse unique à l’ensemble des tables utilisées dans le dashboard.
- Le paramètre `SCOPE` permet de choisir le type de check-in analysé :
    - all : tous les check-ins
    - connect : uniquement Connect
    - mobile : uniquement Mobile

À partir de ce choix :
- `df_scope` : Sous-ensemble cohérent des locations terminées (ended) selon le scope choisi
- `df_gap` :  Locations éligibles à l’analyse du gap entre deux locations (retard et intervalle connus), utilisées pour mesurer :
    - les locations masquées
    - les conflits historiques évités
- `df_next` : Table enrichie avec la location suivante, utilisée pour analyser la propagation des retards

Un affichage de contrôle permet de vérifier rapidement :
- la taille du périmètre
- le nombre de lignes exploitables pour chaque analyse

> Objectif : garantir que toutes les visualisations et KPI reposent sur le même périmètre, sans duplication de logique.
</span>

#### VI. Ponctualité par flux

In [24]:
# Uses the canonical df_ended prepared earlier (ended rentals only).

# Fixed label order + consistent colors across panels
labels = ["En retard", "À l'heure/avance", "Non renseigné"]
colors = ["#FF6B6B", "#3B5BDB", "#74C0FC"]  # late, on-time/early, missing

panels = [
    ("Flux : ALL", get_scoped(df_ended, "all")),
    ("Flux : CONNECT", get_scoped(df_ended, "connect")),
    ("Flux : MOBILE", get_scoped(df_ended, "mobile")),
]

fig = make_subplots(
    rows=1,
    cols=3,
    specs=[[{"type": "pie"}, {"type": "pie"}, {"type": "pie"}]],
    subplot_titles=[title for title, _ in panels],
)

for col_idx, (_, dfi) in enumerate(panels, start=1):
    # Split into 3 mutually exclusive buckets based on delay availability/sign
    count_missing = int(dfi["delay_clipped"].isna().sum())
    count_late = int((dfi["delay_clipped"] > 0).sum())
    count_on_time = int((dfi["delay_clipped"] <= 0).sum())

    fig.add_trace(
        go.Pie(
            labels=labels,
            values=[count_late, count_on_time, count_missing],
            hole=0.40,
            textinfo="percent+label",
            showlegend=(col_idx == 1),  
            marker=dict(colors=colors),
        ),
        row=1,
        col=col_idx,
    )

fig.update_layout(
    height=500,
    width=1600,
    title="Ponctualité par flux : ALL vs CONNECT vs MOBILE (avec Non renseigné)",
    title_x=0.5,
    legend=dict(orientation="h", y=-0.05),
    margin=dict(l=40, r=40, t=80, b=60),
)
fig.show()

# Sanity check: how many ended rentals per check-in flow
print("Distribution checkin_type (ended):")
print(df_ended[COL_CHECKIN].value_counts(dropna=False))


Distribution checkin_type (ended):
checkin_type
mobile     14536
connect     3509
Name: count, dtype: int64


<span style="font-size:0.85em">

Construction de 3 graphiques en anneau (donuts) pour comparer la ponctualité selon le type de check-in :
- ALL : toutes les locations terminées (df_ended)
- CONNECT : uniquement les locations avec check-in Connect
- MOBILE : uniquement les locations avec check-in Mobile

Pour chaque flux, on répartit les locations en 3 catégories exclusives à partir de delay_clipped :
- En retard : delay_clipped > 0
- À l’heure / en avance : delay_clipped <= 0
- Non renseigné : delay_clipped manquant (NaN)

Le graphique utilise :
- un ordre fixe de labels (labels) pour garder une lecture cohérente
- une palette de couleurs constante (colors) entre les 3 donuts
- une légende affichée une seule fois (sur le 1er donut) pour éviter la répétition

À la fin, un sanity check affiche la distribution de checkin_type dans df_ended pour vérifier que les volumes par flux sont cohérents.
</span>

#### VII. Distribution des retards au checkout

In [8]:
# Histogram of positive checkout delays (in minutes), by scope: all / connect / mobile

def plot_delay_histogram(df_ended: pd.DataFrame, scope: str = "all") -> None:
    """Plot the distribution of strictly positive observed delays for a given scope."""

    scope_label = {"all": "Tous flux", "connect": "Connect", "mobile": "Mobile"}.get(scope, scope)

    # Filter to the selected scope (ALL / CONNECT / MOBILE)
    df_scope = get_scoped(df_ended, scope)

    # Keep only observed positive delays
    df_delays = df_scope.loc[
        df_scope["delay_clipped"].notna() & (df_scope["delay_clipped"] > 0),
        ["delay_clipped"],
    ].copy()

    if df_delays.empty:
        print("Aucune ligne éligible pour cet histogramme (retards > 0).")
        return

    # Simple KPIs for the dashboard narrative
    median_delay = float(df_delays["delay_clipped"].median())
    share_over = {t: (df_delays["delay_clipped"] > t).mean() * 100 for t in THRESHOLDS}
    n_rows = len(df_delays)

    fig = px.histogram(
        df_delays,
        x="delay_clipped",
        nbins=NBINS,
        range_x=[0, CLIP_MAX],
        labels={"delay_clipped": "Retard au checkout (minutes, borné)"},
        title=f"Distribution des retards (mn) — {scope_label} (bornage [{CLIP_MIN}, {CLIP_MAX}])",
    )
    fig.update_layout(title_x=0.5, plot_bgcolor="white")

    # Reference lines: median + business thresholds
    fig.add_vline(x=median_delay, line_dash="dash", line_color="black", opacity=0.8)
    fig.add_annotation(
        x=median_delay, y=1.02, xref="x", yref="paper",
        text=f"Médiane ≈ {median_delay:.0f} mn", showarrow=False
    )

    for t in THRESHOLDS:
        fig.add_vline(x=t, line_dash="dot", opacity=0.6)
        fig.add_annotation(x=t, y=1.02, xref="x", yref="paper", text=f"{t} mn", showarrow=False)

    fig.show()

    # Text KPIs (quick read)
    print(f"Lignes (retards > 0) : {n_rows:,}")
    print(f"Médiane du retard    : {median_delay:.1f} mn")
    for t in THRESHOLDS:
        print(f"Part > {t:>3} mn      : {share_over[t]:.1f} %")

    # Context: missing delays inside the same scope
    missing = int(df_scope["delay_clipped"].isna().sum())
    total_scope = len(df_scope)
    print(f"Non renseignés       : {missing:,} ({missing / total_scope * 100:.1f} %)")

plot_delay_histogram(df_ended, scope="all")  # all | connect | mobile


Lignes (retards > 0) : 9,404
Médiane du retard    : 53.0 mn
Part >  60 mn      : 46.6 %
Part > 120 mn      : 27.1 %
Non renseignés       : 1,700 (9.4 %)


<span style="font-size:0.85em">

Définition d'une fonction qui permet de visualiser la distribution des retards positifs (en minutes) au checkout, selon le périmètre choisi :
- all : tous les flux
- connect : check-in Connect uniquement
- mobile : check-in Mobile uniquement

Logique appliquée
- Filtrage par flux: application du périmètre via `get_scoped(df_ended; scope)` afin de garantir la cohérence avec les autres analyses.

- Sélection des retards observés. On conserve uniquement :
    - les lignes avec un `delay_clipped`renseigné
    - et strictement positif (`delay_clipped` > 0) => les vrais retards

- Calcul de KPIs simples
    - médiane du retard
    - part des retards au-delà de seuils business (THRESHOLDS, ex. 60, 120 min)
    - nombre total de retards positifs dans le scope

- Visualisation
    - Histogramme Plotly du retard (borné avec [CLIP_MIN, CLIP_MAX])
    - ligne verticale pour la médiane
    - lignes pointillées pour les seuils métier

- Contexte qualité des données : affichage du nombre et de la part de retards non renseignés pour rappeler les limites du périmètre analyse. 


> Objectif : comprendre la forme de la distribution des retards, identifier les ordres de grandeur dominants et évaluer l’exposition aux retards longs, par type de check-in.
</span>

#### VIII. Impact de la règle "gap" (locations masquées / conflits évités)

In [27]:
def build_gap_curves(df_ended: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """Compute % masked rentals and % avoided historical conflicts as a function of the gap threshold."""
    df_gap = make_df_gap(df_ended).copy()  

    rows = []
    for ci in ("mobile", "connect"):
        sub = df_gap[df_gap[COL_CHECKIN] == ci]

        base_all = len(sub)                      
        base_conflicts = int(sub["was_conflict"].sum())  

        for t in SEUILS:
            masked = int((sub["gap"] < t).sum())
            avoided = int(((sub["gap"] < t) & sub["was_conflict"]).sum())

            rows.append(
                {
                    "Seuil (min)": t,
                    COL_CHECKIN: ci,
                    "% masquées": (masked / base_all * 100) if base_all else 0.0,
                    "% évités": (avoided / base_conflicts * 100) if base_conflicts else 0.0,
                }
            )

    curves = pd.DataFrame(rows)
    loss_long = curves.melt(
        id_vars=["Seuil (min)", COL_CHECKIN],
        value_vars=["% masquées"],
        var_name="metric",
        value_name="value",
    )
    solved_long = curves.melt(
        id_vars=["Seuil (min)", COL_CHECKIN],
        value_vars=["% évités"],
        var_name="metric",
        value_name="value",
    )

    # Prettier legend labels
    loss_long["variable"] = loss_long["metric"] + " " + loss_long[COL_CHECKIN]
    solved_long["variable"] = solved_long["metric"] + " " + solved_long[COL_CHECKIN]

    return loss_long[["Seuil (min)", "variable", "value"]], solved_long[["Seuil (min)", "variable", "value"]], df_gap


def sanity_check_curves(loss_long: pd.DataFrame, solved_long: pd.DataFrame) -> None:
    """Quick checks: 0 at t=0 and monotonic (non-decreasing) by curve."""
    for df_, msg in [
        (loss_long, "À t=0, % masquées devrait être 0."),
        (solved_long, "À t=0, % évités devrait être 0."),
    ]:
        t0 = df_.query("`Seuil (min)` == 0")["value"].sum()
        assert abs(t0) < 1e-9, msg

        for var in df_["variable"].unique():
            y = df_.loc[df_["variable"].eq(var)].sort_values("Seuil (min)")["value"].to_numpy()
            if len(y) > 1:
                assert (np.diff(y) >= -1e-9).all(), f"Non-monotone curve detected: {var}"


def value_at(df_long: pd.DataFrame, label: str, threshold: int) -> float:
    """Extract a metric value at a given threshold (0.0 if missing)."""
    row = df_long[(df_long["Seuil (min)"] == threshold) & (df_long["variable"] == label)]
    return float(row["value"].iloc[0]) if not row.empty else 0.0


# Build curves on the canonical perimeter (ENDED only)
loss_curve_long, solved_curve_long, df_gap = build_gap_curves(df_ended)
sanity_check_curves(loss_curve_long, solved_curve_long)

title_suffix = f"(retard borné [{CLIP_MIN}, {CLIP_MAX}] min, périmètre : ended)"

fig_loss = px.line(
    loss_curve_long,
    x="Seuil (min)",
    y="value",
    color="variable",
    markers=True,
    title=f"🔻 % de locations masquées selon le seuil du gap {title_suffix}",
    labels={
        "Seuil (min)": "Seuil (minutes)",
        "value": "Part (%)",
        "variable": "",
    },
)
fig_loss.update_layout(
    title_x=0.5,
    yaxis_range=[0, 100],
    plot_bgcolor="white",
    legend_title_text="",
)
fig_loss.show()

fig_solved = px.line(
    solved_curve_long,
    x="Seuil (min)",
    y="value",
    color="variable",
    markers=True,
    title=f"✅ % de conflits historiques évités selon le seuil du gap {title_suffix}",
    labels={
        "Seuil (min)": "Seuil (minutes)",
        "value": "Part (%)",
        "variable": "",
    },
)
fig_solved.update_layout(
    title_x=0.5,
    yaxis_range=[0, 100],
    plot_bgcolor="white",
    legend_title_text="",
)
fig_solved.show()

# KPIs at the chosen threshold
masq_mobile = value_at(loss_curve_long, "% masquées mobile", SEUIL_METRIQUE)
masq_connect = value_at(loss_curve_long, "% masquées connect", SEUIL_METRIQUE)
evit_mobile = value_at(solved_curve_long, "% évités mobile", SEUIL_METRIQUE)
evit_connect = value_at(solved_curve_long, "% évités connect", SEUIL_METRIQUE)

print(f"Basé sur {len(df_gap):,} lignes avec 'gap' connu {title_suffix}.")
print(f"Seuil = {SEUIL_METRIQUE} min")
print(f"  Masquées (mobile)   : {masq_mobile:5.2f} %")
print(f"  Masquées (connect)  : {masq_connect:5.2f} %")
print(f"  Évités (mobile)     : {evit_mobile:5.2f} %")
print(f"  Évités (connect)    : {evit_connect:5.2f} %")


Basé sur 1,515 lignes avec 'gap' connu (retard borné [-500, 1000] min, périmètre : ended).
Seuil = 60 min
  Masquées (mobile)   : 21.66 %
  Masquées (connect)  : 22.69 %
  Évités (mobile)     : 59.47 %
  Évités (connect)    : 78.75 %


<span style="font-size:0.85em">

Mesure de l’effet d’une règle produit du type “si le temps entre deux locations (gap) est trop faible, on masque la location”.

L’objectif est double, par flux (mobile vs connect) et pour différents seuils t :
- Locations masquées (%)
    - Règle simulée : une location est masquée si `gap` < t
    - Dénominateur : toutes les locations éligibles (celles avec `delay_clipped` et `gap` connus, mobile/connect)

- Conflits historiques évités (%)
    - Un conflit historique est défini comme : `delay_clipped > gap` (le retard de la location actuelle dépasse le temps dispo avant la suivante → risque de conflit)
    - Un conflit est considéré évité par le seuil t si :
        - c’était un conflit (`was_conflict == True`)
        - et la règle aurait masqué la location (`gap < t`)
    - Dénominateur : les conflits historiques uniquement

Etapes:
1) Construction du dataset “éligible” (`df_gap`)
    - `make_df_gap(df_ended)` conserve :
        - uniquement les locations ended
        - uniquement `checkin_type ∈ {mobile, connect}`
        - uniquement les lignes où `delay_clipped`et `gap` sont disponibles
    - Ajoute `was_conflict = (delay_clipped > gap)`

2) Calcul des courbes (par seuil et par flux)
Pour chaque flux (mobile / connect) et chaque seuil t dans SEUILS :
    - masked = count(gap < t)
    - avoided = count((gap < t) & was_conflict)
    - puis conversion en pourcentage :
        - % masquées = masked / base_all
        - % évités = avoided / base_conflicts

3) Passage au format long + graphes
    - melt() met les résultats au format “long” pour tracer facilement avec Plotly (px.line)
    - Deux courbes tracées :
        - % masquées vs seuil
        - % évités vs seuil

4) Checks de robustesse
    - sanity_check_curves() vérifie :
        - à t=0, les deux % doivent être 0
        - les courbes doivent être monotones croissantes (plus le seuil augmente, plus on masque/évite)

5) KPIs au seuil métier SEUIL_METRIQUE : extrait les valeurs au seuil donné (ex: 60 min) pour alimenter la narration/dashboard.

</span>

#### IX. Propagation du retard vers la location suivante

In [29]:
# Keep ended + relevant flows
ended = df_ended[df_ended[COL_CHECKIN].isin(["mobile", "connect"])].copy()

# Build mapping: previous_ended_rental_id -> next rental_id
# (i.e., "A is previous of B" => map[A] = B)
prev_to_next = (
    ended.dropna(subset=[COL_PREV_ID, COL_RENTAL])
        .astype({COL_PREV_ID: "Int64", COL_RENTAL: "Int64"})
        .set_index(COL_PREV_ID)[COL_RENTAL]
        .to_dict()
)

# Add next_rental_id for each current rental_id
df_next = ended.copy()
df_next[COL_RENTAL] = df_next[COL_RENTAL].astype("Int64")
df_next[COL_NEXT_ID] = df_next[COL_RENTAL].map(prev_to_next)

# Map next delay
delay_map = df_next.set_index(COL_RENTAL)["delay_clipped"].to_dict()
df_next["next_delay"] = df_next[COL_NEXT_ID].map(delay_map)

# Keep valid pairs with both delays known
df_plot = df_next.dropna(subset=["delay_clipped", "next_delay"]).copy()

# Focus on non-negative delays for the next rental + clip for readability
df_plot = df_plot[df_plot["next_delay"] >= 0].copy()
df_plot["next_delay"] = df_plot["next_delay"].clip(0, CLIP_MAX)
df_plot["delay_clipped"] = df_plot["delay_clipped"].clip(CLIP_MIN, CLIP_MAX)

# Scatter
fig_scatter = px.scatter(
    df_plot,
    x="delay_clipped",
    y="next_delay",
    color=COL_CHECKIN,
    color_discrete_map=COLOR_CI,
    labels={
        "delay_clipped": "Retard au checkout de la location actuelle (min, borné)",
        "next_delay": "Retard de la location suivante (min, borné)",
        COL_CHECKIN: "Type de check-in",
    },
    title="Propagation du retard : location actuelle → location suivante (périmètre ended)",
)
fig_scatter.add_hline(y=0, line_dash="dash", opacity=0.6)
fig_scatter.add_vline(x=0, line_dash="dash", opacity=0.6)
fig_scatter.update_layout(title_x=0.5, plot_bgcolor="white")
fig_scatter.show()

# KPIs
n = len(df_plot)
pct_late_next = (df_plot["next_delay"] > 0).mean() * 100 if n else 0.0
avg_next_delay = (
    df_plot.loc[df_plot["next_delay"] > 0, "next_delay"].mean() if n else 0.0
)

print(f"Paires utilisées : {n:,}")
print(f"% de locations suivantes en retard : {pct_late_next:.1f} %")
print(f"Retard moyen de la suivante (conditionnellement aux retards) : {avg_next_delay:.1f} min")


Paires utilisées : 790
% de locations suivantes en retard : 98.2 %
Retard moyen de la suivante (conditionnellement aux retards) : 117.5 min


<span style="font-size:0.85em">

Etude de la répercution d'un retard lors d’une location sur la location suivante du même véhicule (chaîne A → B), sur le périmètre ended uniquement.

*Objectif métier* : Mesurer la propagation opérationnelle :  
- quand une location A est en retard
- est-ce que la location B (la suivante) est plus susceptible d’être en retard ?


*Étapes* : 

1) Restriction du périmètre : Conservation des locations terminées (`df_ended`) et des flux mobile/connect

2) Construction du lien “location suivante”
    - Utilisation de `previous_ended_rental_id` pour retrouver la location suivante :
        - si B a `previous_ended_rental_id` = A
        - alors A → B

    - Construction d'un dictionnaire `prev_to_next` :
        - clé = `previous_ended_rental_id` (location précédente)
        - valeur = `rental_id` (location suivante)

    - Ajout de `next_rental_id` dans `df_next` : identifiant de la location suivante pour chaque location actuelle

3) Récupération du retard de la location suivante
    - Création de `delay_map` : mapping `rental_id → delay_clipped`
    - Calcul de `next_delay` en mappant `next_rental_id` vers ce dictionnaire

4) Sélection des paires exploitables
    - Conservation des lignes où :
        - `delay_clipped` (retard actuel) est connu
        - `next_delay` (retard suivant) est connu
    - Concentration sur `next_delay >= 0` (retards, pas avances) bornage des valeurs pour la lisibilité

5) Visualisation & KPIs
    - Nuage de points : retard actuel (x) vs retard suivant (y), coloré par type de check-in
    - Indicateurs affichés :
        - taille de l’échantillon (nombre de paires)
        - % de locations suivantes en retard
        - retard moyen de la suivante, conditionnellement au fait d’être en retard
</span>

#### X. ROI du seuil : gain marginal & efficacité (résolus / masqués)

In [16]:
# Eligible base: ended rentals with known delay and known gap (mobile/connect only)
imp = df_ended.dropna(subset=["delay_clipped", COL_GAP]).copy()
imp["gap"] = imp[COL_GAP].astype(float)
imp = imp[imp[COL_CHECKIN].isin(["mobile", "connect"])].copy()

# Pre-split by flow (avoid repeating filters in the loop)
imp_mobile = imp[imp[COL_CHECKIN] == "mobile"]
imp_connect = imp[imp[COL_CHECKIN] == "connect"]

def masked_counts(sub: pd.DataFrame, t: int) -> int:
    """Count rentals masked by the product rule (gap < t)."""
    return int((sub["gap"] < t).sum())

def solved_counts(sub: pd.DataFrame, t: int) -> int:
    """Count historical conflicts avoided by threshold t (delay > gap AND gap < t)."""
    was_conflict = sub["delay_clipped"] > sub["gap"]
    return int(((sub["gap"] < t) & was_conflict).sum())

rows_eff, rows_marg, rows_eff_global = [], [], []
prev_solved_total = 0

for t in THRESHOLDS_ROI:
    # Per-flow efficiency
    for ci, sub in (("mobile", imp_mobile), ("connect", imp_connect)):
        solved_ci = solved_counts(sub, t)
        masked_ci = masked_counts(sub, t)
        eff_ci = (solved_ci / masked_ci) if masked_ci > 0 else 0.0
        rows_eff.append({"Seuil (min)": t, "variable": f"Efficacité {ci}", "value": eff_ci})

    # Global (weighted) efficiency + marginal gain
    solved_total = solved_counts(imp, t)
    masked_total = masked_counts(imp, t)
    eff_total = (solved_total / masked_total) if masked_total > 0 else 0.0
    rows_eff_global.append({"Seuil (min)": t, "Efficacité totale": eff_total})

    rows_marg.append({"Seuil (min)": t, "Gain marginal (Total)": solved_total - prev_solved_total})
    prev_solved_total = solved_total

df_eff = pd.DataFrame(rows_eff)
df_marg = pd.DataFrame(rows_marg)
df_eff_total = pd.DataFrame(rows_eff_global)

# Sweet spot: maximize global efficiency (avoid t=0 where masked_total=0)
df_eff_total_nonzero = df_eff_total[df_eff_total["Seuil (min)"] > 0].copy()
sweet_row = df_eff_total_nonzero.loc[df_eff_total_nonzero["Efficacité totale"].idxmax()]
sweet_t = int(sweet_row["Seuil (min)"])
sweet_eff = float(sweet_row["Efficacité totale"])

# Plots
fig_marg = px.line(
    df_marg,
    x="Seuil (min)",
    y="Gain marginal (Total)",
    title="Gain marginal de cas résolus (entre seuils consécutifs)",
    markers=True,
)
fig_marg.add_vline(x=sweet_t, line_dash="dot", line_color="#888")
fig_marg.update_layout(title_x=0.5, plot_bgcolor="white")
fig_marg.show()

fig_eff = px.line(
    df_eff,
    x="Seuil (min)",
    y="value",
    color="variable",
    title="Efficacité par flux = cas résolus / locations masquées",
    markers=True,
)
fig_eff.add_vline(x=sweet_t, line_dash="dot", line_color="#888")
fig_eff.update_layout(
    title_x=0.5,
    yaxis_title="résolus par location masquée",
    plot_bgcolor="white",
    legend_title_text="",
)
fig_eff.show()

fig_eff_total = px.line(
    df_eff_total,
    x="Seuil (min)",
    y="Efficacité totale",
    title="Efficacité totale (pondérée) = résolus_total / masquées_total",
    markers=True,
)
fig_eff_total.add_vline(x=sweet_t, line_dash="dot", line_color="#888")
fig_eff_total.update_layout(title_x=0.5, plot_bgcolor="white")
fig_eff_total.show()

# KPI helper
def pick_metric_eff(df_long: pd.DataFrame, label: str, t: int) -> float:
    """Extract one efficiency value at threshold t."""
    row = df_long[(df_long["Seuil (min)"] == t) & (df_long["variable"] == label)]
    return float(row["value"].iloc[0]) if not row.empty else 0.0

eff_mobile = pick_metric_eff(df_eff, "Efficacité mobile", T_DISPLAY)
eff_connect = pick_metric_eff(df_eff, "Efficacité connect", T_DISPLAY)

print(f"Seuil recommandé (sweet spot) : {sweet_t} min  |  Efficacité totale : {sweet_eff:.3f}")
print(f"[{T_DISPLAY} min]  Efficacité mobile  : {eff_mobile:.3f}  |  connect : {eff_connect:.3f}")
print(f"[{T_DISPLAY} min]  Masquées mobile : {masked_counts(imp_mobile, T_DISPLAY):,}  |  connect : {masked_counts(imp_connect, T_DISPLAY):,}")
print(f"[{T_DISPLAY} min]  Résolus  mobile : {solved_counts(imp_mobile, T_DISPLAY):,}  |  connect : {solved_counts(imp_connect, T_DISPLAY):,}")
print(f"[{T_DISPLAY} min]  Efficacité totale : {(solved_counts(imp, T_DISPLAY)/masked_counts(imp, T_DISPLAY)) if masked_counts(imp, T_DISPLAY)>0 else 0.0:.3f}")


Seuil recommandé (sweet spot) : 15 min  |  Efficacité totale : 0.599
[60 min]  Efficacité mobile  : 0.611  |  connect : 0.420
[60 min]  Masquées mobile : 185  |  connect : 150
[60 min]  Résolus  mobile : 113  |  connect : 63
[60 min]  Efficacité totale : 0.525


<span style="font-size:0.85em">

Identification d'un seuil de gap “intéressant” en comparant :
- le bénéfice : combien de conflits historiques sont évités
- le coût : combien de locations sont masquées par la règle

=> L’idée est de trouver un “sweet spot” : un seuil qui résout beaucoup de conflits pour un minimum de locations masquées.


*Périmètre analysé* : 
Travail sur les locations :
- terminées (ended)
- avec delay_clipped et gap connus
- et sur les flux mobile + connect

Création d'imp découpé en :
- imp_mobile
- imp_connect


*Définitions* :  
1) Locations masquées (coût)
Une location est “masquée” si :
- gap < t (où t est le seuil testé)
- masked_counts(sub, t) renvoie le nombre de locations masquées

2) Conflits historiques évités (bénéfice)
Un conflit historique est défini comme :
- delay_clipped > gap (et donc la location suivante aurait potentiellement un problème)

Un conflit est considéré évité si : conflit historique ET la location serait masquée (gap < t)
- solved_counts(sub, t) renvoie le nombre de conflits évités.

 *Indicateurs calculés pour chaque seuil t (0 → 720 min)* : 
Pour chaque t dans THRESHOLDS_ROI :
- Efficacité par flux : “combien de conflits résolus pour 1 location masquée”
- Efficacité totale pondérée (global) : Même ratio, mais sur l’ensemble du périmètre (mobile+connect), ce qui évite que le flux minoritaire pèse trop dans la décision
- Gain marginal total : “qu’est-ce que j’obtiens en plus quand j’augmente le seuil d’un cran ?”


*Sweet spot (seuil recommandé)* : le seuil t (hors 0) qui maximise l’efficacité totale pondérée  
Cela favorise un seuil “rentable” : beaucoup de conflits évités pour peu de masquage.


*Visualisations* :  
- Gain marginal vs seuil : Permet de voir à partir de quand l’augmentation du seuil apporte peu de bénéfice supplémentaire
- Efficacité par flux (mobile vs connect) : Comparaison des performances par type de check-in
- Efficacité totale (pondérée) : Indicateur global utilisé pour choisir le seuil recommandé

Une ligne verticale (sweet_t) est ajoutée pour matérialiser le seuil “optimal” sur les graphes.

*KPIs affichés en sortie* : 
- seuil recommandé + efficacité totale associée
- efficacité mobile et connect à T_DISPLAY
- nombres masqués / résolus à T_DISPLAY
- efficacité totale à T_DISPLAY

> Résumé métier :
Cette partie met en balance coût (locations masquées) vs bénéfice (conflits évités) pour identifier un seuil de gap qui maximise l’impact opérationnel tout en limitant la perte potentielle de disponibilité.

#### XI. Scénarios business : estimation du CA impacté (proxy)

In [17]:
def run_business_scenario(
    df_ended: pd.DataFrame,
    threshold_min: int = 60,
    avg_duration_days: float = 1.5,
    loss_rate: float = 0.60,          # share of masked rentals that translate into lost revenue (proxy)
    mean_daily_price: float = 120.0,  # €/day (use pricing-derived value)
    scope: str = "all",               # all | connect | mobile
    verbose: bool = True,
) -> dict:
    """Compute dashboard-ready business KPIs for a given gap threshold (proxy model)."""

    df = get_scoped(df_ended, scope)

    eligible = df.dropna(subset=["delay_clipped", COL_GAP]).copy()
    if eligible.empty:
        if verbose:
            print("Aucune ligne éligible (delay/gap manquants).")
        return {}

    eligible["gap"] = eligible[COL_GAP].astype(float)

    # Product rule proxy: rental is "masked" if gap < threshold
    masked = eligible["gap"] < threshold_min
    n_eligible = len(eligible)
    n_masked = int(masked.sum())
    pct_masked = (n_masked / n_eligible * 100) if n_eligible else 0.0

    # Historical conflicts (delay > gap) and avoided conflicts if masked
    conflict_before = eligible["delay_clipped"] > eligible["gap"]
    avoided = conflict_before & masked

    n_conflicts = int(conflict_before.sum())
    n_avoided = int(avoided.sum())
    pct_avoided = (n_avoided / n_conflicts * 100) if n_conflicts else 0.0

    # Revenue proxy computed on this perimeter only (not global business GMV)
    baseline_proxy = n_eligible * mean_daily_price * avg_duration_days
    lost_proxy = n_masked * loss_rate * mean_daily_price * avg_duration_days
    share_proxy = (lost_proxy / baseline_proxy * 100) if baseline_proxy else 0.0

    result = {
        "scope": scope,
        "threshold_min": threshold_min,
        "n_eligible": n_eligible,
        "n_masked": n_masked,
        "pct_masked": pct_masked,
        "n_conflicts": n_conflicts,
        "n_avoided": n_avoided,
        "pct_avoided": pct_avoided,
        "avg_duration_days": avg_duration_days,
        "loss_rate": loss_rate,
        "mean_daily_price": mean_daily_price,
        "lost_revenue_proxy_eur": lost_proxy,
        "baseline_revenue_proxy_eur": baseline_proxy,
        "share_revenue_proxy_pct": share_proxy,
    }

    if verbose:
        print(f"=== Scénario business (scope='{scope}') ===")
        print(f"- Seuil (gap)                 : {threshold_min} min")
        print(f"- Lignes éligibles            : {n_eligible:,}")
        print(f"- Locations masquées (proxy)  : {n_masked:,}  ({pct_masked:.1f} %)")
        print(f"- Conflits historiques        : {n_conflicts:,}")
        print(f"- Conflits évités (proxy)     : {n_avoided:,}  ({pct_avoided:.1f} %)")
        print(f"- Hypothèses CA → durée={avg_duration_days} j, perte={loss_rate*100:.0f} %, prix/j={mean_daily_price:.0f} €")
        print(f"- Part CA affectée (proxy)    : {share_proxy:.1f} %")
        print(f"- CA perdu (proxy)            : {lost_proxy:,.0f} € | baseline (proxy) {baseline_proxy:,.0f} €")

    return result


# Run scenarios (example)
_ = run_business_scenario(
    df_ended,
    threshold_min=60,
    avg_duration_days=1.5,
    loss_rate=0.60,
    mean_daily_price=121,
    scope="all",
)

_ = run_business_scenario(
    df_ended,
    threshold_min=60,
    avg_duration_days=1.5,
    loss_rate=0.60,
    mean_daily_price=121,
    scope="connect",
)


=== Scénario business (scope='all') ===
- Seuil (gap)                 : 60 min
- Lignes éligibles            : 1,515
- Locations masquées (proxy)  : 335  (22.1 %)
- Conflits historiques        : 270
- Conflits évités (proxy)     : 176  (65.2 %)
- Hypothèses CA → durée=1.5 j, perte=60 %, prix/j=121 €
- Part CA affectée (proxy)    : 13.3 %
- CA perdu (proxy)            : 36,482 € | baseline (proxy) 274,972 €
=== Scénario business (scope='connect') ===
- Seuil (gap)                 : 60 min
- Lignes éligibles            : 661
- Locations masquées (proxy)  : 150  (22.7 %)
- Conflits historiques        : 80
- Conflits évités (proxy)     : 63  (78.8 %)
- Hypothèses CA → durée=1.5 j, perte=60 %, prix/j=121 €
- Part CA affectée (proxy)    : 13.6 %
- CA perdu (proxy)            : 16,335 € | baseline (proxy) 119,972 €


<span style="font-size:0.85em">

Transformation des résultats “opérationnels” (gap, retards, conflits) en une estimation business :
combien de chiffre d’affaires (CA) pourrait être affecté si on applique un seuil de gap (buffer).

> Important : il s’agit d’un proxy (estimation simplifiée), calculé uniquement sur le périmètre éligible (locations enchaînées avec gap connu), pas sur tout le business.

*Objectif* : Pour un seuil donné threshold_min (ex. 60 min) et un scope (all, connect, mobile), la fonction calcule :  
- combien de locations seraient masquées par la règle  
- combien de conflits historiques seraient évités  
- une estimation du CA potentiellement perdu (proxy)  


*Périmètre et filtrage* : 
- Application du scope via get_scoped()  
- Conservation des lignes éligibles :  
    - delay_clipped connu  
    - gap connu
- Convertion du gap en numérique pour sécuriser les calculs


*Règle produit simulée (coût)* : Une location est considérée masquée si :
- gap < threshold_min
On en déduit :
- n_masked : nombre de locations masquées
- pct_masked : proportion de locations masquées sur le périmètre éligible

Interprétation : perte de disponibilité / friction induite par la règle.


*Conflits historiques évités (bénéfice)* : 
- Définition du conflit historique : 
    - delay_clipped > gap
- Un conflit est évité (proxy) si :
    - c’était un conflit et
    - la location est masquée (gap < threshold_min)

Calcul :
- n_conflicts : nombre de conflits historiques
- n_avoided : nombre de conflits évités par la règle
- pct_avoided : part des conflits évités  
Interprétation : gain opérationnel / réduction du risque.

*Estimation CA (proxy)* : Hypothèses business : 
- avg_duration_days : durée moyenne d’une location (ex. 1.5 jours)
- mean_daily_price : prix moyen journalier (issu du dataset pricing ou fixé)
- loss_rate : part des locations masquées qui se traduirait effectivement par une perte de CA (proxy)

Calculs :
- baseline_proxy : CA théorique sur le périmètre éligible
- lost_proxy : CA estimé “perdu” à cause des locations masquées
- share_proxy : part de CA affectée sur ce périmètre  

Interprétation : ordre de grandeur du coût business associé à un seuil.

*Sortie* : La fonction renvoie un dictionnaire result contenant :
- KPIs volume (% masquées, % évités)
- hypothèses (prix, durée, loss_rate)
- estimations CA (proxy)

Si verbose=True, un résumé lisible est imprimé pour faciliter la narration en soutenance.
</span>